In [26]:
from zipfile import ZipFile
file_name= "albeniz.zip"

with ZipFile (file_name,'r') as zip:
  zip.extractall()
  print("DONE")
  
  
  

DONE


In [0]:
 #!rm -rf "m"

In [0]:
class Song:
    def __init__(self, midi, max_instrument_note_length, max_vel, max_pitch, max_duration):
        self.midi = midi
        self.max_vel = max_vel
        self.max_pitch = max_pitch
        self.max_duration = max_duration
        self.max_instrument_note_length = max_instrument_note_length


In [0]:
import pretty_midi
import numpy as np
# from song import Song
from sklearn.preprocessing import OneHotEncoder
# import tensorflow as tf


class MIDIProcessor:

    def __init__(self, size):
        self.all_songs_objects = np.zeros((size, 1))
        self.all_songs_objects = np.empty((0, 1))
        self.size = size
        self.one_hot_encoder = OneHotEncoder(categories='auto')
        self.one_hot_encoder.fit(np.linspace(0, 443, 444).reshape(-1, 1))
        self.final_network_input = np.zeros((size, 1))
        self.final_network_input = np.empty((0, 1))
        self.max_param_song=Song(0,0,0,0,0)

    def read_files(self, start_file, end_file):
        print("start = ", start_file, "end =", end_file)
        for i in range(start_file+1, end_file):
            try:
                song = pretty_midi.PrettyMIDI("alb_esp{}.mid".format(i))
            except:
                break

            maxVel, maxPitch, maxDuration = 0.0, 0.0, 0.0
            max_instrument_note_length = 0
            for instrument in song.instruments:
                if len(instrument.notes) > max_instrument_note_length:
                    max_instrument_note_length = len(instrument.notes)
            for instrument in song.instruments:
                if len(instrument.notes) == max_instrument_note_length:
                    for note in instrument.notes:
                        duration = note.end - note.start
                        if (duration > maxDuration):
                            maxDuration = duration
                        if (note.velocity > maxVel):
                            maxVel = note.velocity
                        if (note.pitch > maxPitch):
                            maxPitch = note.pitch
                    break

            new_song = Song(song, max_instrument_note_length, maxVel, maxPitch, maxDuration)
            self.all_songs_objects = np.append(self.all_songs_objects, [new_song])

            if(new_song.max_duration>self.max_param_song.max_duration):
                self.max_param_song.max_duration=new_song.max_duration  #TODO : save the max between all songs in max param song 
            if( maxPitch > self.max_param_song.max_pitch):

                self.max_param_song.max_pitch= maxPitch
            if(new_song.max_vel > self.max_param_song.max_vel):

                self.max_param_song.max_vel=new_song.max_vel



    def one_hot_encode(self, song):
        indices = self.encode_song(song)[:60]
        print(indices)
        n_labels = len(indices)
        n_unique_labels = 444
        one_hot_encoded_song = np.zeros((n_labels, n_unique_labels))
        one_hot_encoded_song[np.arange(n_labels), indices] = 1

        return one_hot_encoded_song

    def encode_song(self, song):
        encoded_song = []
        max_instrument_note_length = 0
        for instrument in song.midi.instruments:
            if len(instrument.notes) > max_instrument_note_length:
                max_instrument_note_length = len(instrument.notes)
        for instrument in song.midi.instruments:
            if (instrument.name == "Drum Kit 2"):
                continue
            if not instrument.is_drum:
                if len(instrument.notes) == max_instrument_note_length:  # choose the most effective instrument
                    for n in instrument.notes:
                        encoded_song.append(self.label_encoder(n, song))
                    break
        return encoded_song

    # TODO: Check SONG OR ALL?  -> yes doing that
    def label_encoder(self, note, song):
        d = self.max_param_song.max_duration / 4 # instead od song.... use max param
        DurationLabel = int((note.end - note.start) / d)
        v = self.max_param_song.max_vel / 4
        VelocityLabel = int(note.velocity / v)
        p = self.max_param_song.max_pitch / 4
        PitchLabel = int(note.pitch / p)
        return DurationLabel * 10 + VelocityLabel * 1 + PitchLabel*100

    def decode_song(self, note): # use max param to reconvert it 
        # cello_c=pretty_midi.PrettyMIDI()

        # cello_prog=pretty_midi.instrument_name_to_program("Electric Grand Piano")

        # cello0=pretty_midi.instrument(program=cello_prog)
        # point=0
        # for 
        pass


    def write(self, song):
        pass


In [32]:

import keras 
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.layers import Dropout
import numpy as np

#while using keras we dont need to get batch out of data by ourself , keras has an ready syntax for that

# batch_processor = BatchProcessor(batch_size=2, dataset_size=2)
# batch=batch_processor.get_next_batch()

midiprocessor = MIDIProcessor(7)

midiprocessor.read_files(0,7)
encoded_data = []
print(len(midiprocessor.all_songs_objects))
for item in midiprocessor.all_songs_objects:
    encoded_data.append(midiprocessor.one_hot_encode(item))

encoded_data=np.array(encoded_data)    
print(encoded_data)



#training model

#Encoder
encoder_inputs = Input(shape=(None,encoded_data.shape[2]))
endocer_lstm1=LSTM(1024,return_sequences=True)
encoder_output=endocer_lstm1(encoder_inputs)
#Dropout

encoder_output= Dropout(0.3)(encoder_output)

encoder_lstm2=LSTM(1024,return_state=True)

encoder_output,encoder_h,encoder_c=encoder_lstm2(encoder_output)

encoder_states=[encoder_h,encoder_c]
#Decoder
decoder_input=Input(shape=(None,encoded_data.shape[2]))
decoder_lstm1=LSTM(1024,return_sequences=True)
decoder_output=decoder_lstm1(decoder_input,initial_state=encoder_states)#seeting the decoder intial state to encoder states
decoder_lstm2=LSTM(1024,return_sequences=True,return_state=True)
decoder_out,_,_=decoder_lstm2(decoder_output)
decoder_dence=Dense(encoded_data.shape[2],activation='softmax')
decoder_out=decoder_dence(decoder_out)


# #define model
model=Model(inputs=[encoder_inputs,decoder_input],outputs=[decoder_out])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit(x=[encoded_data,encoded_data],y=encoded_data,batch_size=6,epochs=300) #TO DO: add validation split while having more data



# inference model
encoder_model=Model(encoder_inputs,encoder_states)

decoder_state_input_h=Input(shape=(1024,))
decoder_state_input_c=Input(shape=(1024,))
decoder_states_input=[decoder_state_input_h,decoder_state_input_c]
decoder_output=decoder_lstm1(decoder_input,initial_state=decoder_states_input)
decoder_output, state_h, state_c=decoder_lstm2(decoder_output)
decoder_states=[state_h,state_c]
# dence - softmax layer
decoder_output=decoder_dence(decoder_output)

decoder_model=Model([decoder_input]+decoder_states_input, [decoder_output]+decoder_states)



start=np.random.randint(0,len(encoded_data))
states_value=encoder_model.predict(encoded_data[start].reshape(1,60,444))
#predict recursively
target_seq=np.zeros((1,1,encoded_data.shape[2]))
target_seq[0,0,0]=1 # set the first note

decoded=[]
for i in range (60):
    output_tkn,h,c=decoder_model.predict([target_seq]+states_value)
    sample=np.argmax(output_tkn[0,-1,:])
    decoded.append(sample)
    target_seq=np.zeros((1,1,encoded_data.shape[2]))
    target_seq[0,0,sample]=1 # get the next from the prev one in testing/infering
    states_value=[h,c]
print(decoded)




















start =  0 end = 7
6
[302, 312, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 202, 202, 202, 202, 202, 202, 202, 202, 202, 302, 302, 302, 302, 302, 202, 202, 202, 202, 202, 202, 201, 201, 201, 302, 312, 302, 301, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 302, 301]
[202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 201, 201, 202, 202, 202, 202, 202, 212, 211, 211, 211, 202, 203, 202, 203, 202, 202, 203, 212, 202, 201, 202, 212, 212, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202]
[202, 202, 202, 202, 202, 202, 202, 302, 202, 202, 202, 202, 202, 202, 202, 302, 202, 202, 202, 202, 202, 202, 202, 302, 202, 202, 202, 202, 202, 202, 202, 302, 202, 202, 202, 202, 202, 202, 202, 302, 202, 202, 202, 202, 202, 202, 202, 302, 202, 202, 202, 202, 302, 302, 203, 302, 302, 303, 303, 303]
[302, 202, 202, 302, 202, 302, 302, 302, 302, 202, 302, 302, 202, 302, 302, 